In [1]:
import os
import time
import json
import random
from pathlib import Path
from functools import partial

from urllib.parse import urlencode, urlparse, urlunparse, parse_qs, unquote

import pandas as pd
import numpy as np

from omegaconf import OmegaConf

from transformations.id_rules import IDRules

import ipyplot as iplt
import matplotlib.pyplot as plt

pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

%load_ext autoreload
%autoreload 2

In [3]:
data_dir = Path("./data/")
config = OmegaConf.load("./conf/config.yaml")

In [4]:
merchant = "excell_sports"
feed_url = config.feedurl[merchant]
feed_url

'https://feeds.performancehorizon.com/aisl3/1100l944/2a0a29f0760f1a96cc0bf9bb513fbb3e.csv.zip'

In [14]:
df = pd.read_csv(feed_url, compression="gzip" if feed_url.endswith("/gzip/") else "zip", low_memory=False)
df

,ID,TITLE,DESCRIPTION,GOOGLE PRODUCT CATEGORY,LINK,ADWORDS REDIRECT,IMAGE,CONDITION,AVAILABILITY,BRAND,ONLINE ONLY,PRICE,MPN,GTIN,SHIPPING NAME,DELIVERY PRICE,PRODUCT TYPE,GENDER,AGE GROUP,COLOR,SIZE,Unnamed: 21
0,440603,New Balance Mens Heat Grid 1/2 Zip Size: Small...,The New Balance Mens Heat Grid 1/2 Zip in thi...,Sporting Goods > Exercise & Fitness,https://prf.hn/click/camref:1011lozwx/creative...,https://prf.hn/click/camref:1011lozwx/creative...,https://excell-sports.com/images/new-balance-m...,new,in stock,New Balance,N,69.95 GBP,65421046003,1.959073e+11,UK Standard Delivery - Royal Mail Tracked48,4.99,Men > Clothing > Men's Hoodies & Jumpers,NaN,NaN,Light Blue,Small,NaN
1,440604,New Balance Mens Heat Grid 1/2 Zip Size: Mediu...,The New Balance Mens Heat Grid 1/2 Zip in th...,Sporting Goods > Exercise & Fitness,https://prf.hn/click/camref:1011lozwx/creative...,https://prf.hn/click/camref:1011lozwx/creative...,https://excell-sports.com/images/new-balance-m...,new,in stock,New Balance,N,69.95 GBP,65421046004,1.959073e+11,UK Standard Delivery - Royal Mail Tracked48,4.99,Men > Clothing > Men's Hoodies & Jumpers,NaN,NaN,Light Blue,Medium,NaN
2,440605,New Balance Mens Heat Grid 1/2 Zip Size: Large...,The New Balance Mens Heat Grid 1/2 Zip in th...,Sporting Goods > Exercise & Fitness,https://prf.hn/click/camref:1011lozwx/creative...,https://prf.hn/click/camref:1011lozwx/creative...,https://excell-sports.com/images/new-balance-m...,new,in stock,New Balance,N,69.95 GBP,65421046005,1.959073e+11,UK Standard Delivery - Royal Mail Tracked48,4.99,Men > Clothing > Men's Hoodies & Jumpers,NaN,NaN,Light Blue,Large,NaN
3,440606,New Balance Mens Heat Grid 1/2 Zip Size: Extra...,The New Balance Mens Heat Grid 1/2 Zip in th...,Sporting Goods > Exercise & Fitness,https://prf.hn/click/camref:1011lozwx/creative...,https://prf.hn/click/camref:1011lozwx/creative...,https://excell-sports.com/images/new-balance-m...,new,in stock,New Balance,N,69.95 GBP,65421046006,1.959073e+11,UK Standard Delivery - Royal Mail Tracked48,4.99,Men > Clothing > Men's Hoodies & Jumpers,NaN,NaN,Light Blue,Extra Large,NaN
4,440591,New Balance Mens Graphic Impact Run Packable J...,The New Balance Mens Graphic Impact Run Packa...,Sporting Goods > Exercise & Fitness,https://prf.hn/click/camref:1011lozwx/creative...,https://prf.hn/click/camref:1011lozwx/creative...,https://excell-sports.com/images/new-balance-m...,new,in stock,New Balance,N,79.95 GBP,65420506003,1.959073e+11,UK Standard Delivery - Royal Mail Tracked48,4.99,Sport > Running > Running Clothing,NaN,NaN,Eclipse,Small,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29049,70,"Adidas Copa Mundial Size: UK 10, Colour: Black",The adidas Copa Mundial in the classic Black...,Sporting Goods > Exercise & Fitness,https://prf.hn/click/camref:1011lozwx/creative...,https://prf.hn/click/camref:1011lozwx/creative...,https://excell-sports.com/images/adidas-copa-m...,new,in stock,Adidas,N,129.95 GBP,447021009,4.003421e+12,UK Standard Delivery - Royal Mail Tracked48,0.00,Sport > Football > Football Boots,NaN,NaN,Black,UK 10,NaN
29050,71,"Adidas Copa Mundial Size: UK 10.5, Colour: Black",The adidas Copa Mundial in the classic Black...,Sporting Goods > Exercise & Fitness,https://prf.hn/click/camref:1011lozwx/creative...,https://prf.hn/click/camref:1011lozwx/creative...,https://excell-sports.com/images/adidas-copa-m...,new,in stock,Adidas,N,129.95 GBP,447021010,4.003421e+12,UK Standard Delivery - Royal Mail Tracked48,0.00,Sport > Football > Football Boots,NaN,NaN,Black,UK 10.5,NaN
29051,72,"Adidas Copa Mundial Size: UK 11, Colour: Black",The adidas Copa Mundial in the classic Black...,Sporting Goods > Exercise & Fitness,https://prf.hn/click/camref:1011lozwx/creative...,https://prf.hn/click/camref:1011lozwx/creative...,https://excell-sports.com/images/adidas-copa-m...,new,in stock,Adidas,N,129.95 GBP,447021011,4.003421e+12,UK Standard Delivery - Royal Mail Tracked48,0.00,Sport > Footbal

In [15]:
df.to_parquet(data_dir / "raw" / f"{merchant}.parquet")

In [16]:
id_rule = IDRules(config, merchant)
df = df.dropna(subset=[config.pid[merchant]]).reset_index(drop=True)
df["id"] = df.apply(id_rule, axis=1)
df = df.dropna(subset=["id"]).reset_index(drop=True)
df

,ID,TITLE,DESCRIPTION,GOOGLE PRODUCT CATEGORY,LINK,ADWORDS REDIRECT,IMAGE,CONDITION,AVAILABILITY,BRAND,ONLINE ONLY,PRICE,MPN,GTIN,SHIPPING NAME,DELIVERY PRICE,PRODUCT TYPE,GENDER,AGE GROUP,COLOR,SIZE,Unnamed: 21,id
0,440603,New Balance Mens Heat Grid 1/2 Zip Size: Small...,The New Balance Mens Heat Grid 1/2 Zip in thi...,Sporting Goods > Exercise & Fitness,https://prf.hn/click/camref:1011lozwx/creative...,https://prf.hn/click/camref:1011lozwx/creative...,https://excell-sports.com/images/new-balance-m...,new,in stock,New Balance,N,69.95 GBP,65421046003,1.959073e+11,UK Standard Delivery - Royal Mail Tracked48,4.99,Men > Clothing > Men's Hoodies & Jumpers,NaN,NaN,Light Blue,Small,NaN,excell_sports.p34969
1,440604,New Balance Mens Heat Grid 1/2 Zip Size: Mediu...,The New Balance Mens Heat Grid 1/2 Zip in th...,Sporting Goods > Exercise & Fitness,https://prf.hn/click/camref:1011lozwx/creative...,https://prf.hn/click/camref:1011lozwx/creative...,https://excell-sports.com/images/new-balance-m...,new,in stock,New Balance,N,69.95 GBP,65421046004,1.959073e+11,UK Standard Delivery - Royal Mail Tracked48,4.99,Men > Clothing > Men's Hoodies & Jumpers,NaN,NaN,Light Blue,Medium,NaN,excell_sports.p34969
2,440605,New Balance Mens Heat Grid 1/2 Zip Size: Large...,The New Balance Mens Heat Grid 1/2 Zip in th...,Sporting Goods > Exercise & Fitness,https://prf.hn/click/camref:1011lozwx/creative...,https://prf.hn/click/camref:1011lozwx/creative...,https://excell-sports.com/images/new-balance-m...,new,in stock,New Balance,N,69.95 GBP,65421046005,1.959073e+11,UK Standard Delivery - Royal Mail Tracked48,4.99,Men > Clothing > Men's Hoodies & Jumpers,NaN,NaN,Light Blue,Large,NaN,excell_sports.p34969
3,440606,New Balance Mens Heat Grid 1/2 Zip Size: Extra...,The New Balance Mens Heat Grid 1/2 Zip in th...,Sporting Goods > Exercise & Fitness,https://prf.hn/click/camref:1011lozwx/creative...,https://prf.hn/click/camref:1011lozwx/creative...,https://excell-sports.com/images/new-balance-m...,new,in stock,New Balance,N,69.95 GBP,65421046006,1.959073e+11,UK Standard Delivery - Royal Mail Tracked48,4.99,Men > Clothing > Men's Hoodies & Jumpers,NaN,NaN,Light Blue,Extra Large,NaN,excell_sports.p34969
4,440591,New Balance Mens Graphic Impact Run Packable J...,The New Balance Mens Graphic Impact Run Packa...,Sporting Goods > Exercise & Fitness,https://prf.hn/click/camref:1011lozwx/creative...,https://prf.hn/click/camref:1011lozwx/creative...,https://excell-sports.com/images/new-balance-m...,new,in stock,New Balance,N,79.95 GBP,65420506003,1.959073e+11,UK Standard Delivery - Royal Mail Tracked48,4.99,Sport > Running > Running Clothing,NaN,NaN,Eclipse,Small,NaN,excell_sports.p34968
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29049,70,"Adidas Copa Mundial Size: UK 10, Colour: Black",The adidas Copa Mundial in the classic Black...,Sporting Goods > Exercise & Fitness,https://prf.hn/click/camref:1011lozwx/creative...,https://prf.hn/click/camref:1011lozwx/creative...,https://excell-sports.com/images/adidas-copa-m...,new,in stock,Adidas,N,129.95 GBP,447021009,4.003421e+12,UK Standard Delivery - Royal Mail Tracked48,0.00,Sport > Football > Football Boots,NaN,NaN,Black,UK 10,NaN,excell_sports.
29050,71,"Adidas Copa Mundial Size: UK 10.5, Colour: Black",The adidas Copa Mundial in the classic Black...,Sporting Goods > Exercise & Fitness,https://prf.hn/click/camref:1011lozwx/creative...,https://prf.hn/click/camref:1011lozwx/creative...,https://excell-sports.com/images/adidas-copa-m...,new,in stock,Adidas,N,129.95 GBP,447021010,4.003421e+12,UK Standard Delivery - Royal Mail Tracked48,0.00,Sport > Football > Football Boots,NaN,NaN,Black,UK 10.5,NaN,excell_sports.
29051,72,"Adidas Copa Mundial Size: UK 11, Colour: Black",The adidas Copa Mundial in the classic Black...,Sporting Goods > Exercise & Fitness,https://prf.hn/click/camref:1011lozwx/creative...,https://prf.hn/click/camref:1011lozwx/creative...,https://excell-sports.com/im

In [17]:
df[config.pid[merchant]].value_counts().sum() == df.id.value_counts().sum()

True

In [18]:
df.id.value_counts()

excell_sports.          484
excell_sports.p7961      41
excell_sports.p3794      34
excell_sports.p3816      32
excell_sports.p1693      27
                       ... 
excell_sports.p31157      1
excell_sports.p24760      1
excell_sports.p33302      1
excell_sports.p20110      1
excell_sports.p27358      1
Name: id, Length: 5736, dtype: int64

In [19]:
df.merchant_category.value_counts().head(100)

AttributeError: 'DataFrame' object has no attribute 'merchant_category'

In [ ]:
df.merchant_product_category_path.value_counts().head(30)

In [24]:
(data_dir / "filtered").mkdir(exist_ok=True)

def filter_categories(df, category_col, category_vals):
    print("category col:", category_col)
    print(df[category_col].value_counts().head(20))
    print("category vals:")
    print(category_vals)
    print()
    print("rows before: ", len(df))
    df = df.loc[df[category_col].isin(set(category_vals))].reset_index(drop=True)
    print("rows after: ", len(df))
    print()
    print("cols retained:")
    print(df[category_col].value_counts())
    return df

df = filter_categories(df, config.filter[merchant].col, config.filter[merchant].vals)
if "col2" in config.filter[merchant]:
    df = filter_categories(df, config.filter[merchant].col2, config.filter[merchant].vals2)
df.to_parquet(data_dir / "filtered" / f"{merchant}.parquet")

category col: PRODUCT TYPE
Sport > Running > Running Footwear                    1787
Sport > Football > Football Boots                     1727
Men > Clothing > Men's T-Shirts & Polo Shirts         1073
Men > Footwear > Men's Trainers                        927
Sport > Football > Football Training Clothing          892
Juniors > Boys Footwear > Trainers For Boys            813
Sport > Training & Fitness > Gym Clothing              805
Men > Clothing > Men's Tracksuit Bottoms               775
Women > Footwear > Women's Trainers                    771
Men > Clothing > Men's Hoodies & Jumpers               687
Sport > Other Sports > Outdoor > Jackets & Gilets      671
Juniors > Girls Footwear > Trainers For Girls          670
Women > Clothing > Women's T-Shirts & Polo Shirts      665
Sport > Swim > Swim Wear                               592
Sport > Other Sports > Outdoor > Footwear              591
Men > Clothing > Men's Jackets & Gilets                486
Sport > Running > Running Clo

In [28]:
df.id.value_counts()

excell_sports.          64
excell_sports.p13437    27
excell_sports.p13438    22
excell_sports.p31099    19
excell_sports.p18972    18
                        ..
excell_sports.p22482     1
excell_sports.p22763     1
excell_sports.p32187     1
excell_sports.p16329     1
excell_sports.p25053     1
Name: id, Length: 779, dtype: int64

In [29]:
df.id.nunique()

779

In [30]:
df.brand_name.value_counts().head(40)

AttributeError: 'DataFrame' object has no attribute 'brand_name'

In [31]:
df.colour.value_counts().head(50)

AttributeError: 'DataFrame' object has no attribute 'colour'

In [32]:
for col in df.columns:
    print(col)
    print("-" * 100)
    print(df[col].value_counts().head(20))
    print("-" * 100)

ID
----------------------------------------------------------------------------------------------------
439862    1
332259    1
332257    1
332256    1
332255    1
332254    1
332253    1
332252    1
332251    1
332250    1
332249    1
332271    1
332270    1
332269    1
332268    1
332267    1
333415    1
333414    1
333413    1
333412    1
Name: ID, dtype: int64
----------------------------------------------------------------------------------------------------
TITLE
----------------------------------------------------------------------------------------------------
EA7 Mens Trainers Size: UK 11, Colour: Black                   6
EA7 Mens Trainers Size: UK 9, Colour: Black                    6
EA7 Mens Trainers Size: UK 10, Colour: Black                   5
EA7 Mens Trainers Size: UK 8, Colour: Black                    5
Adidas Womens Courtpoint Shoes Size: UK 4, Colour: White       3
Puma Mens Flyer Flex Running Shoe Size: UK 7, Colour: Black    3
Puma Mens Flyer Flex Running Shoe S

In [33]:
random.choices(df.DESCRIPTION.values, k=5)

['  The adidas Womens VL Court Shoes in this Tech Indigo with Footwear White and Matte Silver colourway remix classic style so you can wear them every single day. A suede upper provides a soft feel, and the textile lining and cushioned sockliner give you all-day comfort.      Lace closure    Suede upper    Enjoy the comfort and performance of OrthoLite Float sockliner    ',
 '  The adidas Womens VL Court Shoes in this Mauve with Footwear White colourway. These skate-inspired shoes remix classic style so you can wear them every single day. A suede upper provides a soft feel, and the textile lining and cushioned sockliner give you all-day comfort.      Lace closure    Suede upper    Enjoy the comfort and performance of OrthoLite Float sockliner    ',
 ' The EA7 Mens Sea World Slides in this White colourway. Slides with a decorated oversized band featuring the EA7 maxi logo. A versatile style, perfect for the pool or the beach and for taking with you to the gym.      Band with contrasting

In [ ]:
random.choices(df.product_short_description.values, k=5)